This script is designed to process text files in a structured manner. It begins by loading the necessary data and defining agents and tools required for the task. These agents and tools are configured to work sequentially, ensuring that the text files located in the root directory are properly loaded. Once the text files are loaded, the script processes their content by resuming (extracting key points) from each file. Finally, it generates a concise summary of all the texts, providing a high-level overview of the information contained in the files.

In [1]:
import sys
import os
from dotenv import load_dotenv

src_path = os.path.abspath(os.path.join(os.getcwd(), "..", "src"))
print("Adding src folder to path:", src_path)
sys.path.insert(0, src_path)

load_dotenv(f"{src_path}/.env")

Adding src folder to path: c:\Users\ricar\Github\augumented-rag\src


True

### Loading Models from the Source Folder

To begin, ensure that the required models are loaded from the source folder. This step is crucial for initializing the necessary components for processing and analysis. Verify that the source folder contains all the relevant model files before proceeding.

In [2]:
from app.schemas.models import VideoData, AudioData, ImageData, TextData

### Loading Data Models

In the next cell, we will load the data models corresponding to various data sources: audio, text, video, and image. These models represent the structured data from the respective folders inside the `./data` directory. Each model is designed to handle the specific characteristics of its data type, ensuring efficient processing and analysis.

In [3]:
audio_data_1 = AudioData(
    source="./data/audios/azure-podcast-2.mp3",
    objective="Explains Azure ACA and its features",
    tags=["azure", "aca", "containers"]
)

audio_data_2 = AudioData(
    source="./data/audios/azure-podcast-2.mp3",
    objective="Explains Azure in general",
    tags=["azure", "definitions"]
)

print("AudioData instance:", audio_data_1)
print("AudioData instance:", audio_data_2)

AudioData instance: source='./data/audios/azure-podcast-2.mp3' objective='Explains Azure ACA and its features' tags=['azure', 'aca', 'containers'] encoding=None embeddings=None
AudioData instance: source='./data/audios/azure-podcast-2.mp3' objective='Explains Azure in general' tags=['azure', 'definitions'] encoding=None embeddings=None


In [4]:
text_data_1 = TextData(
    source="./data/documents/aoai-assistants.pdf",
    objective="Explains Azure OpenAI and its features",
    tags=["azure", "aoai", "assistants"]
)

text_data_2 = TextData(
    source="./data/documents/aoai-prompting.pdf",
    objective="Explains Prompting in Azure OpenAI",
    tags=["azure", "aoai", "prompting"]
)

print("TextData instance:", text_data_1)
print("TextData instance:", text_data_2)

TextData instance: source='./data/documents/aoai-assistants.pdf' objective='Explains Azure OpenAI and its features' tags=['azure', 'aoai', 'assistants'] encoding=None embeddings=None
TextData instance: source='./data/documents/aoai-prompting.pdf' objective='Explains Prompting in Azure OpenAI' tags=['azure', 'aoai', 'prompting'] encoding=None embeddings=None


### Loading Toolers and Creating the Assembly

In this step, we focus on loading the appropriate toolers required for processing the data. Toolers are specialized components designed to handle specific tasks, such as extracting embeddings, encoding data, or performing analysis. Once the toolers are loaded, they are assembled into a cohesive workflow to ensure seamless processing of the data models (audio, video, image, and text).

The assembly of toolers ensures that each data type is processed using the most suitable tools, leveraging their unique capabilities to extract meaningful insights and achieve the desired objectives.

In [5]:
import uuid
from app.schemas.models import Agent, Assembly

audio_agent = Agent(
    id=str(uuid.uuid4()),
    name="AudioAgent",
    model_id="default",
    metaprompt="This agent handles audio processing and extraction tasks.",
    objective="audio"
)

text_agent = Agent(
    id=str(uuid.uuid4()),
    name="TextAgent",
    model_id="default",
    metaprompt="This agent analyzes textual information for semantic understanding.",
    objective="text"
)

assembly = Assembly(
    id=str(uuid.uuid4()),
    objective="multimodal processing using local data for architecture review on azure",
    agents=[audio_agent, text_agent],
    roles=["audio", "text"]
)


print("Assembly created with agents:", assembly)

Assembly created with agents: id='8868e737-56b2-4991-bd3e-943de429d3df' objective='multimodal processing using local data for architecture review on azure' agents=[Agent(id='6a286f12-6309-4ca4-a764-b5543f5ce625', name='AudioAgent', model_id='default', metaprompt='This agent handles audio processing and extraction tasks.', objective='audio'), Agent(id='f9192c13-d801-4cb7-8354-97d91147a6d9', name='TextAgent', model_id='default', metaprompt='This agent analyzes textual information for semantic understanding.', objective='text')] roles=['audio', 'text']


In [6]:
from app.agents.main import ToolerOrchestrator

orchestrator = ToolerOrchestrator()

In [11]:
response = await orchestrator.run_interaction(
    assembly=assembly,
    prompt="""
        You have been given a few local documents and audios on the folder located at the directory 'C:\\Users\\ricar\\Github\\augumented-rag\\notebook\\data'
        and contains the subfolders 'documents' and 'audios'. I need a abstract detailing their content.
        Preciselly give the name of the Python Classes used as Tools on your answer.
    """,
    strategy="parallel"
)

flattened = [
    str(item)
    for sublist in response
    for item in (sublist if isinstance(sublist, list) else [sublist])
]

print("Orchestrator response:", "\n".join(flattened))

Erro durante a transcrição de áudio: Exception with error code: 
[CALL STACK BEGIN]

    > pal_string_to_wstring
    - pal_string_to_wstring
    - pal_string_to_wstring
    - pal_string_to_wstring
    - pal_string_to_wstring
    - pal_string_to_wstring
    - pal_string_to_wstring
    - pal_string_to_wstring
    - pal_string_to_wstring
    - pal_string_to_wstring
    - pal_string_to_wstring
    - pal_string_to_wstring
    - pal_string_to_wstring
    - pal_string_to_wstring
    - pal_string_to_wstring
    - recognizer_create_speech_recognizer_from_config

[CALL STACK END]

Exception with an error code: 0x8 (SPXERR_FILE_OPEN_FAILED)


Orchestrator response: It seems you've sent an empty prompt. Could you please provide more details or let me know how I can assist you?
I encountered issues while processing the documents and audios:

1. **Text Processing**: The summarization service is currently not configured, so I couldn't generate abstracts for the documents (`aoai-assistants.pdf`, `aoai-prompting.pdf`, `aoai.pdf`).

2. **Audio Processing**: There was an error with the audio transcription due to a file access problem (`SPXERR_FILE_OPEN_FAILED`). Therefore, I couldn't extract details from the audio files.

If you can reconfigure the services or provide alternate methods, please let me know so I can proceed effectively. Additionally, I could manually edit, review, or process any content based on further instructions.


In [10]:
response = await orchestrator.run_interaction(assembly=assembly, prompt="Explain how may I use Azure OpenAI to build a chatbot on AKS or ACA", strategy="llm")
flattened = [str(item) for sublist in response for item in (sublist if isinstance(sublist, list) else [sublist])]
print("Orchestrator response:", "\n".join(flattened))

Orchestrator response: Tools available:
• Text extraction and analysis tool (to extract content from Microsoft Azure documentation and related references).

Using Reference: Microsoft Azure documentation on Azure OpenAI, AKS, and ACA

Answer:
You can build a chatbot that leverages Azure OpenAI models on container orchestration platforms like Azure Kubernetes Service (AKS) or Azure Container Apps (ACA) by following these general steps:

1. Provision the Azure OpenAI Service:
  • Create an Azure OpenAI resource in your subscription via the Azure portal.
  • Retrieve your API endpoint and key. These credentials are required so your chatbot code can authenticate and interact with the chosen model (for example, ChatGPT or another language model).

2. Develop Your Chatbot Application:
  • Write your chatbot code (in Python, Node.js, etc.) so that it calls the Azure OpenAI endpoint. Your application should format user inputs into API requests and then process the responses.
  • Implement stat

In [9]:
#response = await orchestrator.run_interaction(assembly=assembly, prompt="Detail each information that I have to consider on Azure Container Instance to Run a multi-agentic RAG app", strategy="llm")
#flattened = [str(item) for sublist in response for item in (sublist if isinstance(sublist, list) else [sublist])]
#print("Orchestrator response:", "\n".join(flattened))